In [6]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as emb
from dwave.samplers import SimulatedAnnealingSampler
import dimod
from scipy.sparse import lil_matrix, csc_matrix, block_diag

In [5]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
import psutil
import time
from scipy.sparse import block_diag
import matplotlib.pyplot as plt

# Optimized function
def generate_hamiltonian_optimized(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    modules = sorted(event.modules, key=lambda module: module.z)
    
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)
    A_ang_blocks = []
    A_bif_blocks = []
    A_inh_blocks = []
    b = np.zeros(N)

    block_size = 500
    num_blocks = (N + block_size - 1) // block_size

    for block_idx in range(num_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, N)

        A_ang_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_bif_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_inh_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)

        for i in range(start_idx, end_idx):
            seg_i = segments[i]
            vect_i = seg_i.to_vect()
            norm_i = np.linalg.norm(vect_i)

            for j in range(i + 1, end_idx):
                seg_j = segments[j]
                vect_j = seg_j.to_vect()
                norm_j = np.linalg.norm(vect_j)

                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)
                if np.abs(cosine - 1) < 1e-9:
                    A_ang_block[i - start_idx, j - start_idx] = 1
                    A_ang_block[j - start_idx, i - start_idx] = 1

                if seg_i.from_hit == seg_j.from_hit and seg_i.to_hit != seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha

                if seg_i.from_hit != seg_j.from_hit and seg_i.to_hit == seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha

                s_ab = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
                if s_ab > 0:
                    A_inh_block[i - start_idx, j - start_idx] = beta * s_ab * s_ab
                    A_inh_block[j - start_idx, i - start_idx] = beta * s_ab * s_ab

        A_ang_blocks.append(A_ang_block)
        A_bif_blocks.append(A_bif_block)
        A_inh_blocks.append(A_inh_block)

    A_ang = block_diag(A_ang_blocks, format='csc')
    A_bif = block_diag(A_bif_blocks, format='csc')
    A_inh = block_diag(A_inh_blocks, format='csc')

    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments

# Non-optimized function
def generate_hamiltonian(event, params):
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    modules = copy.deepcopy(event.modules)
    modules.sort(key=lambda a: a.z)

    segments = [em.segment(from_hit, to_hit) for idx in range(len(modules) - 1) for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)]
    N = len(segments)
    A = np.zeros((N, N))
    A_ang = np.zeros((N, N))
    A_bif = np.zeros((N, N))
    
    b = np.zeros(N)

    s_ab = np.zeros((N, N))
    for i, seg_i in enumerate(segments):
        for j, seg_j in enumerate(segments):
            s_ab[i, j] = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
    A_inh = np.zeros((N, N))

    for i, seg_i in enumerate(segments):
        for j, seg_j in enumerate(segments):
            if i != j:
                vect_i = seg_i.to_vect()
                vect_j = seg_j.to_vect()
                cosine = np.dot(vect_i, vect_j) / (np.linalg.norm(vect_i) * np.linalg.norm(vect_j))

                eps = 1e-9

                if np.abs(cosine - 1) < eps:
                    A_ang[i, j] = 1

                if (seg_i.from_hit == seg_j.from_hit) and (seg_i.to_hit != seg_j.to_hit):
                    A_bif[i, j] = -alpha

                if (seg_i.from_hit != seg_j.from_hit) and (seg_i.to_hit == seg_j.to_hit):
                    A_bif[i, j] = -alpha

                A_inh[i, j] = s_ab[i, j] * s_ab[j, i] * beta

    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments

# Performance test
def performance_test(event, params):
    process = psutil.Process()

    print("Running performance test for optimized function...")
    start_memory = process.memory_info().rss / (1024 ** 2)
    start_time = time.time()
    A_optimized, b_optimized, segments_optimized = generate_hamiltonian_optimized(event, params)
    end_memory = process.memory_info().rss / (1024 ** 2)
    end_time = time.time()
    memory_used_optimized = end_memory - start_memory
    time_taken_optimized = end_time - start_time

    print(f"Optimized: Memory used: {memory_used_optimized:.2f} MB, Time taken: {time_taken_optimized:.6f} seconds")

    print("Running performance test for non-optimized function...")
    start_memory = process.memory_info().rss / (1024 ** 2)
    start_time = time.time()
    A_non_optimized, b_non_optimized, segments_non_optimized = generate_hamiltonian(event, params)
    end_memory = process.memory_info().rss / (1024 ** 2)
    end_time = time.time()
    memory_used_non_optimized = end_memory - start_memory
    time_taken_non_optimized = end_time - start_time

    print(f"Non-optimized: Memory used: {memory_used_non_optimized:.2f} MB, Time taken: {time_taken_non_optimized:.6f} seconds")

    return memory_used_optimized, time_taken_optimized, memory_used_non_optimized, time_taken_non_optimized

# Test for increasing matrix size
def run_scaling_tests():
    params = {
        'alpha': 1.0,
        'beta': 1.0,
        'lambda': 100.0,
    }

    N_PARTICLES_LIST = [5,10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # Test different event sizes to increase matrix size
    memory_optimized_list = []
    time_optimized_list = []
    memory_non_optimized_list = []
    time_non_optimized_list = []

    # Define your detector and event generation here as per the earlier setup
    N_MODULES = 5
    LX = float("+inf")
    LY = float("+inf")
    Z_SPACING = 1.0

    detector = toy.SimpleDetectorGeometry(
        module_id=list(range(N_MODULES)),
        lx=[LX] * N_MODULES,
        ly=[LY] * N_MODULES,
        z=[i + Z_SPACING for i in range(N_MODULES)]
    )

    generator = toy.SimpleGenerator(
        detector_geometry=detector,
        theta_max=np.pi / 6
    )

    for N_PARTICLES in N_PARTICLES_LIST:
        event = generator.generate_event(N_PARTICLES)
        memory_opt, time_opt, memory_non_opt, time_non_opt = performance_test(event, params)

        # Append the results
        memory_optimized_list.append(memory_opt)
        time_optimized_list.append(time_opt)
        memory_non_optimized_list.append(memory_non_opt)
        time_non_optimized_list.append(time_non_opt)

    return N_PARTICLES_LIST, memory_optimized_list, time_optimized_list, memory_non_optimized_list, time_non_optimized_list

# Plotting function
def plot_performance(N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized):
    plt.figure(figsize=(10, 5))

    # Plot Time taken
    # Plot Time taken
    plt.subplot(1, 2, 1)
    plt.plot(N_PARTICLES_LIST, time_optimized, label='Optimized', marker='o')
    plt.plot(N_PARTICLES_LIST, time_non_optimized, label='Non-Optimized', marker='o')
    plt.xlabel('Number of Particles (Increasing Matrix Size)')
    plt.ylabel('Time Taken (seconds)')
    plt.title('Time vs Matrix Size')
    plt.legend()

    # Plot Memory usage
    plt.subplot(1, 2, 2)
    plt.plot(N_PARTICLES_LIST, memory_optimized, label='Optimized', marker='o')
    plt.plot(N_PARTICLES_LIST, memory_non_optimized, label='Non-Optimized', marker='o')
    plt.xlabel('Number of Particles (Increasing Matrix Size)')
    plt.ylabel('Memory Used (MB)')
    plt.title('Memory vs Matrix Size')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Run the scaling tests and plot results
N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized = run_scaling_tests()
plot_performance(N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized)


Running performance test for optimized function...
Optimized: Memory used: 0.00 MB, Time taken: 0.184077 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 0.00 MB, Time taken: 0.266390 seconds
Running performance test for optimized function...
Optimized: Memory used: 0.52 MB, Time taken: 1.001028 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 1.22 MB, Time taken: 2.666614 seconds
Running performance test for optimized function...
Optimized: Memory used: 4.16 MB, Time taken: 4.066929 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 19.52 MB, Time taken: 40.253318 seconds
Running performance test for optimized function...
Optimized: Memory used: 3.71 MB, Time taken: 9.655838 seconds
Running performance test for non-optimized function...


KeyboardInterrupt: 

In [ ]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
import psutil
import time
from scipy.sparse import block_diag, lil_matrix
import matplotlib.pyplot as plt

# Optimized and non-optimized functions (unchanged)

# Performance test with error handling
def performance_test(event, params):
    process = psutil.Process()

    # Optimized function
    print("Running performance test for optimized function...")
    try:
        start_memory = process.memory_info().rss / (1024 ** 2)
        start_time = time.time()
        A_optimized, b_optimized, segments_optimized = generate_hamiltonian_optimized(event, params)
        end_memory = process.memory_info().rss / (1024 ** 2)
        end_time = time.time()
        memory_used_optimized = end_memory - start_memory
        time_taken_optimized = end_time - start_time
        print(f"Optimized: Memory used: {memory_used_optimized:.2f} MB, Time taken: {time_taken_optimized:.6f} seconds")
    except MemoryError:
        print("Optimized function ran out of memory!")
        memory_used_optimized = None
        time_taken_optimized = None

    # Non-optimized function
    print("Running performance test for non-optimized function...")
    try:
        start_memory = process.memory_info().rss / (1024 ** 2)
        start_time = time.time()
        A_non_optimized, b_non_optimized, segments_non_optimized = generate_hamiltonian(event, params)
        end_memory = process.memory_info().rss / (1024 ** 2)
        end_time = time.time()
        memory_used_non_optimized = end_memory - start_memory
        time_taken_non_optimized = end_time - start_time
        print(f"Non-optimized: Memory used: {memory_used_non_optimized:.2f} MB, Time taken: {time_taken_non_optimized:.6f} seconds")
    except MemoryError:
        print("Non-optimized function ran out of memory!")
        memory_used_non_optimized = None
        time_taken_non_optimized = None

    return memory_used_optimized, time_taken_optimized, memory_used_non_optimized, time_taken_non_optimized

# Test for increasing matrix size with error handling
def run_scaling_tests():
    params = {
        'alpha': 1.0,
        'beta': 1.0,
        'lambda': 100.0,
    }

    N_PARTICLES_LIST = [2, 10, 20, 30, 40, 50 , 60 , 70, 80, 90, 100]  # Test different event sizes to increase matrix size
    memory_optimized_list = []
    time_optimized_list = []
    memory_non_optimized_list = []
    time_non_optimized_list = []

    # Define your detector and event generation here as per the earlier setup
    N_MODULES = 5
    LX = float("+inf")
    LY = float("+inf")
    Z_SPACING = 1.0

    detector = toy.SimpleDetectorGeometry(
        module_id=list(range(N_MODULES)),
        lx=[LX] * N_MODULES,
        ly=[LY] * N_MODULES,
        z=[i + Z_SPACING for i in range(N_MODULES)]
    )

    generator = toy.SimpleGenerator(
        detector_geometry=detector,
        theta_max=np.pi / 6
    )

    for N_PARTICLES in N_PARTICLES_LIST:
        event = generator.generate_event(N_PARTICLES)
        memory_opt, time_opt, memory_non_opt, time_non_opt = performance_test(event, params)

        # Append the results (even if None, it will be handled in plotting)
        memory_optimized_list.append(memory_opt)
        time_optimized_list.append(time_opt)
        memory_non_optimized_list.append(memory_non_opt)
        time_non_optimized_list.append(time_non_opt)

    return N_PARTICLES_LIST, memory_optimized_list, time_optimized_list, memory_non_optimized_list, time_non_optimized_list

# Plotting function with error handling for missing data
def plot_performance(N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized):
    plt.figure(figsize=(10, 5))

    # Plot Time taken
    plt.subplot(1, 2, 1)
    plt.plot(N_PARTICLES_LIST, time_optimized, label='Optimized', marker='o', linestyle='--')
    if any(time_non_optimized):
        plt.plot(N_PARTICLES_LIST, time_non_optimized, label='Non-Optimized', marker='o', linestyle='--')
    plt.xlabel('Number of Particles (Increasing Matrix Size)')
    plt.ylabel('Time Taken (seconds)')
    plt.title('Time vs Matrix Size')
    plt.legend()

    # Plot Memory usage
    plt.subplot(1, 2, 2)
    plt.plot(N_PARTICLES_LIST, memory_optimized, label='Optimized', marker='o', linestyle='--')
    if any(memory_non_optimized):
        plt.plot(N_PARTICLES_LIST, memory_non_optimized, label='Non-Optimized', marker='o', linestyle='--')
    plt.xlabel('Number of Particles (Increasing Matrix Size)')
    plt.ylabel('Memory Used (MB)')
    plt.title('Memory vs Matrix Size')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Run the scaling tests and plot results
N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized = run_scaling_tests()
plot_performance(N_PARTICLES_LIST, memory_optimized, time_optimized, memory_non_optimized, time_non_optimized)


Running performance test for optimized function...
Optimized: Memory used: 0.00 MB, Time taken: 0.011413 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 0.00 MB, Time taken: 0.012102 seconds
Running performance test for optimized function...
Optimized: Memory used: 0.00 MB, Time taken: 1.302001 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 1.22 MB, Time taken: 3.251372 seconds
Running performance test for optimized function...
Optimized: Memory used: 0.20 MB, Time taken: 4.398766 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 19.54 MB, Time taken: 43.429212 seconds
Running performance test for optimized function...
Optimized: Memory used: 7.46 MB, Time taken: 10.435627 seconds
Running performance test for non-optimized function...
Non-optimized: Memory used: 96.03 MB, Time taken: 238.579614 seconds
Running performance test for optimized function...
Optimiz